# Gunshot Detection Model for Raspberry Pi Pico

This notebook implements a machine learning model for gunshot detection using TensorFlow, which will be deployed on a Raspberry Pi Pico RP2040 microcontroller using TensorFlow Lite Micro.

In [9]:
import os
import numpy as np
import pandas as pd
import librosa
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

print(f'TensorFlow version: {tf.__version__}')


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/lib/python3/dist-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/mounith/.local/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/usr/lib/python3/dist-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
  File "/usr/lib/python3/dist-packages/tornado/platform/asyncio

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



ImportError: numpy.core.multiarray failed to import

In [ ]:
def extract_features(file_path, duration=5):
    try:
        # Load the audio file
        y, sr = librosa.load(file_path, duration=duration)
        
        # Extract features
        # 1. MFCCs (Mel-frequency cepstral coefficients)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        mfcc_mean = np.mean(mfccs, axis=1)
        
        # 2. Spectral Centroid
        spectral_centroids = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
        sc_mean = np.mean(spectral_centroids)
        
        # 3. Zero Crossing Rate
        zero_crossings = librosa.feature.zero_crossing_rate(y)[0]
        zc_mean = np.mean(zero_crossings)
        
        # 4. Spectral Rolloff
        spectral_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)[0]
        sr_mean = np.mean(spectral_rolloff)
        
        # Combine all features
        features = np.concatenate([
            mfcc_mean,
            [sc_mean, zc_mean, sr_mean]
        ])
        
        return features
        
    except Exception as e:
        print(f'Error extracting features from {file_path}: {str(e)}')
        return None

In [ ]:
def create_model(input_shape):
    # Create a lightweight model suitable for microcontroller
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(input_shape,)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    
    # Compile the model
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [ ]:
def plot_training_history(history):
    plt.figure(figsize=(12, 4))
    
    # Plot accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

In [ ]:
def convert_to_tflite(model):
    # Convert the model to TensorFlow Lite format
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    
    # Enable quantization
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.inference_input_type = tf.int8
    converter.inference_output_type = tf.int8
    
    # Convert the model
    tflite_model = converter.convert()
    
    # Save the model
    with open('gunshot_detection_model.tflite', 'wb') as f:
        f.write(tflite_model)
    
    return tflite_model

def convert_tflite_to_c_array(tflite_model):
    # Convert model to C array
    c_array = []
    for byte in tflite_model:
        c_array.append(f'0x{byte:02x}')
    
    # Create C source code
    c_code = 'const unsigned char model_data[] = {\n  '
    c_code += ',\n  '.join([', '.join(c_array[i:i+12]) for i in range(0, len(c_array), 12)])
    c_code += '\n};\n'
    c_code += f'const unsigned int model_data_len = {len(tflite_model)};\n'
    
    # Save to file
    with open('model_data.h', 'w') as f:
        f.write(c_code)
    
    return c_code

In [ ]:
# Configure Kaggle credentials
os.environ['KAGGLE_USERNAME'] = 'YOUR_KAGGLE_USERNAME'  # Replace with your Kaggle username
os.environ['KAGGLE_KEY'] = 'YOUR_KAGGLE_API_KEY'  # Replace with your Kaggle API key

# Create directories if they don't exist
os.makedirs('./datasets/gunshots', exist_ok=True)

# Download and extract the gunshot dataset
!kaggle datasets download -d emrahaydemr/gunshot-audio-dataset -p ./datasets/gunshots
!unzip ./datasets/gunshots/gunshot-audio-dataset.zip -d ./datasets/gunshots/

# Define paths for gunshot and non-gunshot audio files
gunshot_path = './datasets/gunshots/Gunshot sound'
nongunshot_path = './datasets/gunshots/Background sound'

# Process both gunshot and non-gunshot files
print('Processing gunshot files...')
gunshot_features, gunshot_labels = process_directory(gunshot_path, 1)  # 1 for gunshot

print('Processing non-gunshot files...')
nongunshot_features, nongunshot_labels = process_directory(nongunshot_path, 0)  # 0 for non-gunshot

# Combine datasets
X = np.array(gunshot_features + nongunshot_features)
y = np.array(gunshot_labels + nongunshot_labels)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(f'Training dataset shape: {X_train.shape}')
print(f'Testing dataset shape: {X_test.shape}')
print(f'Number of gunshot samples: {sum(y == 1)}')
print(f'Number of non-gunshot samples: {sum(y == 0)}')

# Create and train the model
model = create_model(X_train.shape[1])

# Training parameters
epochs = 50
batch_size = 32

# Add early stopping to prevent overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_split=0.2,
    callbacks=[early_stopping]
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'\nTest accuracy: {test_accuracy:.4f}')

# Plot training history
plot_training_history(history)

# Save the trained model
model.save('gunshot_detection_model.h5')
print('\nModel saved as gunshot_detection_model.h5')

# Convert to TFLite format
tflite_model = convert_to_tflite(model)
print('Model converted to TFLite format')

# Convert to C array for Raspberry Pi Pico
c_code = convert_tflite_to_c_array(tflite_model)
print('Model converted to C array format for Raspberry Pi Pico deployment')